# Postype 크롤링 
> 포스타입은 최근 소설을 읽을 때 가장 많이 이용하는 플랫폼이다. 창작물을 볼 때도 있지만, 원작이 있는 작품의 패러디물을 보는 경우가 많고 전체 작품 중에서 그러한 비중도 상당하다.

평소 많이 이용하다 보니 코드를 통해 포스타입 크롤링을 해보고 싶다는 생각이 들었다. 만들어보고 싶은 것들은 다음과 같다. 

- 인기순으로 작품 제목들과 그 하트 수를 크롤링하여 데이터로 저장한다.
- 인기순으로 작품 제목, 하트, 작가, 업로드 날짜를 크롤링하여 데이터로 저장한다.
- 인기순으로 작품 제목들을 크롤링하여 많이 언급되는 단어들을 뽑아 워드 클라우드로 만든다. 
- 성인 작품들만 작품 제목, 하트, 작가, 업로드 날짜를 크롤링해 데이터로 저장한다.
- 인기 많은 작품을 중 성인 작품들만 골라내어 제목과 해당 링크를 출력한다. 

## 1.TEST_BeautifulSoup
> 인기순으로 게시물을 재정렬하려면 필연적으로 가상 브라우저를 사용하여야 한다. 하지만 여기서는 연습 삼아 현재 페이지를 기준으로 크롤링을 진행해보았다. 크롤링한 항목은 다음과 같다. 

- 작품 제목
- 작가
- 블로그 url
- 좋아요 개수(하트 수) 

성인용 작품까지 크롤링하려면 포스타입에 로그인이 되어 있고, 성인인증이 끝난 상태여야 한다!

In [1]:
# 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen
import pandas as pd

In [2]:
# URL 분석
# 'https://www.postype.com/search/index?keyword=' + 검색어


def title_crawling(couple_name):
    
    url = 'https://www.postype.com/search?keyword=' + couple_name
    
    # 크롤링 함수 만들기 
    html=requests.get(url).content
    soup=BeautifulSoup(html, 'html.parser')
    # print(soup)
    
    res=soup.find_all('h3', {'class': 'pst-title'}) # 제목 텍스트를 담고 있는 a 태그의 부모 h3 태그 전부 꺼내기
    res2=soup.find_all('div', {'class' : 'pst-action'}) # 좋아요 수를 담은 span 태그를 답고 있는 div 태그 전부 꺼내기
    res3=soup.find_all('div', {'class' : 'pst-blog'})


    # 제목 리스트, 글 링크 리스트 생성
    title_li=[]
    url_li=[]

    for l in res:
        title=l.get_text() # 제목 텍스트 추출
        text_url=l.find('a')['href'] # 글 링크 추출
        title_li.append(title) # 제목 리스트에 추가
        url_li.append(text_url) # 링크 리스트에 추가

    print(title_li)
    title_li = [t.strip() for t in title_li] # 좌우 공백 삭제
    
    
    # 작가 리스트 생성
    writer_li = []
    
    for w in res3:
        writer_name=w.find('a').get_text() # a 태그를 찾지 않고 텍스트를 추출할 경우 블로그 이름까지 추출된다
        writer_li.append(writer_name)
    
    writer_li=[x.strip() for x in writer_li]

        
    # 하트 수 리스트 생성
    like_li=[]
    for h in res2:
        like_count = h.find('span', {'class' : 'count'})
        
        if like_count!=None: # 좋아요가 있는 게시물
            like_count=int(like_count.get_text())
            like_li.append(like_count)
        
        else: # 좋아요가 없는 게시물
            like_li.append(0) # 0을 추가한다
            

    # 리스트 값 확인    
    # print(title_li)
    # print(url_li)
    # print(like_li)
    # print(writer_li)

    
    # 데이터 프레임으로 만들기
    df = pd.DataFrame(data={'title' : title_li, 
                            'writer' : writer_li,
                            'url' : url_li,
                           'likes' : like_li})

    df.to_csv('postype6.csv') # 글자가 깨지지 않게 하려면 인코딩을 cp949로 해야 한다!
    
    return print('문서 저장이 완료되었습니다.')


# 검색어(커플명) 입력
couple_name = input()
title_crawling(couple_name)

금자희재
['\nI am completely, totally, utterly yours, at your service.\n', '\n[금자희재] 일상의새벽\n', '\n[금자희재]회자정리(會者定離)-3\n', '\n[금자희재] If The World Was Ending\n', '\n무제 (11)\n', '\n[금자희재] Blood//Water\n', '\n[연홍이창/금자희재] 인因(5)\n', '\n2020-03 백업 05\n', '\n[금자희재] Our Shining Days\n', '\n[금자희재/희재금자] 난파선 下\n', '\n[금자희재/희재금자] 난파선 上\n', '\n놀이터\n']
문서 저장이 완료되었습니다.


## 예시

In [56]:
# 저장된 데이터 불러오기
df=pd.read_csv('postype7.csv', index_col=0)
df

,title,writer,url,likes
0,나미 생일 계획기,Bae,https://baefrombest.postype.com/post/7210953,1
1,[나미른] 썰 백업 (20.05.19~06.24),벨,https://r1ngmabell.postype.com/post/6883418,5
2,내가 파는 원피스 커플링에 대한 개인적 견해.,A,https://start-a.postype.com/post/7104233,1
3,[나미른] 썰 백업 (20.04.17~05.18),벨,https://r1ngmabell.postype.com/post/6620888,3
4,루나미 첫사랑,카시와,https://kil1the1132.postype.com/post/6809724,2
5,[루나미] 잘 부탁드립니다,벨,https://r1ngmabell.postype.com/post/6546014,7
6,[루나미] 밀짚모자의 주인,벨,https://r1ngmabell.postype.com/post/6549806,9
7,NOTICE,라포,https://rapport-zs.postype.com/post/6742755,0
8,[루나미] 0505,𝕍𝐄𝐑𝐘🍓𝔹𝐄𝐑𝐑𝐘,https://ifff8e7i.postype.com/post/6731125,2
9,[루나미] 좋은 노랠 들려줄게,벨,https://r1ngmabell.postype.com/post/6637870,6


## 2. TEST_Selenium 
> 이번엔 selenium과 Chromedriver를 이용하여 인기순으로 작품을 재정렬하고, 특정 페이지까지의 작품을 크롤링해보자.

BeautifulSoup과 마찬가지로 성인용 작품까지 포함해 모든 작품을 크롤링하려면 포스타입에 성인 인증이 끝난 상태로 로그인 되어있어야 한다!

### 인기순으로 작품 재정렬하기

In [12]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select # dropdown 선택을 위해 필수!
import time

In [14]:
# 검색어 설정
couple_name='시목여진'
url = 'https://www.postype.com/search?keyword=' + couple_name

# 드라이버 설정
driver=webdriver.Chrome('chromedriver.exe') # chrome X Chrome O
# driver=webdriver.Chrome('chrome_driver.exe') # chrome X Chrome O
driver.get(url)

time.sleep(2)

select = Select(driver.find_element_by_id('search-sort')) # 드롭다운 메뉴 선택 
select.select_by_value('popularity') # 인기순 선택 

driver.find_element_by_xpath('//*[@id="search-tools"]/div/div[4]/button').click() # 필터링 적용 버튼 클릭

time.sleep(2)
    
driver.quit()

인기순으로 작품을 재정렬하는 것도 해결했다. 그렇다면 이제는 본격적으로 작품들을 크롤링해볼 차례다! 

## 3. BeautifulSoup + Selenium
### 인기 작품 제목, 작가, 좋아요 수 크롤링
위의 두 코드를 참고하여 인기 작품들의 제목과 작가, 글 링크와 좋아요 수까지 크롤링해보자. 

In [22]:
couple_name = '금자희재'
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.postype.com/search?page={}&keyword='+ couple_name +'&period=any_time&sort=popularity&options_title=1&options_sub_title=1&options_content=1&options_tags=1&options_nickname=1'

# 넘길 페이지 수 지정
page = 5

# 데이터프레임을 생성하기 위한 리스트 생성
title_li=[] # 글 제목
url_li=[] # 글 URL
writer_li = [] # 작가 이름
like_li=[] # 좋아요 수 


# 크롤링 함수 만들기 

for i in range(1, page):
    link = url.format(i)
    driver.get(link)
    
    time.sleep(3)
    
    html=requests.get(link).content
    soup=BeautifulSoup(html, 'html.parser')
    # print(soup)

    res=soup.find_all('h3', {'class': 'pst-title'}) # 제목 텍스트를 담고 있는 a 태그의 부모 h3 태그 전부 꺼내기
    res2=soup.find_all('div', {'class' : 'pst-action'}) # 좋아요 수를 담은 span 태그를 답고 있는 div 태그 전부 꺼내기
    res3=soup.find_all('div', {'class' : 'pst-blog'})

    for l in res:
        title=l.get_text() # 제목 텍스트 추출
        text_url=l.find('a')['href'] # 글 링크 추출
        title_li.append(title) # 제목 리스트에 추가
        url_li.append(text_url) # 링크 리스트에 추가
    
    print('{} 페이지 작품 제목 리스트는 {}입니다.'.format(i, title_li))

    for w in res3:
        writer_name=w.find('a').get_text() # a 태그를 찾지 않고 텍스트를 추출할 경우 블로그 이름까지 추출된다
        writer_li.append(writer_name)

    for h in res2:
        like_count = h.find('span', {'class' : 'count'})

        if like_count!=None: # 좋아요가 있는 게시물
            like_count=int(like_count.get_text())
            like_li.append(like_count)

        else: # 좋아요가 없는 게시물
            like_li.append(0) # 0을 추가한다

            
# 리스트 값들의 좌우 공백 삭제            
title_li = [t.strip() for t in title_li] 
writer_li=[x.strip() for x in writer_li]

print('크롤링이 완료되었습니다. 데이터로 저장합니다.')
driver.quit() # 가상 브라우저 종료 


# 데이터 프레임으로 만들기
df = pd.DataFrame(data={'title' : title_li, 
                    'writer' : writer_li,
                    'url' : url_li,
                   'likes' : like_li})

df.to_csv('postype_crawling.csv', encoding='cp949') # 글자가 깨지지 않게 하려면 인코딩을 cp949로 해야 한다!
print('데이터 저장이 완료되었습니다.')

1 페이지 작품 제목 리스트는 ['\n하이에나 (1)\n', '\n[금자희재] HB\n', '\n[금자희재/희재금자] 비포 선라이즈\n', '\n[금자희재/희재금자] 세상에서 제일 어려운 건\n', '\n충忠\n', '\n[금자희재/희재금자] 0310\n', '\n[금자희재] 드레스를 입은 밤\n', '\n[금자희재/희재금자] 사계 (1)\n', '\n금자의 세계\n', '\n[연홍이창/금자희재] 인因 (1)\n', '\n[금자희재/희재금자] 내 상사가 사람을 꼬시려고 해 (1)\n', '\n[금자희재] Daybreak\n']입니다.
2 페이지 작품 제목 리스트는 ['\n하이에나 (1)\n', '\n[금자희재] HB\n', '\n[금자희재/희재금자] 비포 선라이즈\n', '\n[금자희재/희재금자] 세상에서 제일 어려운 건\n', '\n충忠\n', '\n[금자희재/희재금자] 0310\n', '\n[금자희재] 드레스를 입은 밤\n', '\n[금자희재/희재금자] 사계 (1)\n', '\n금자의 세계\n', '\n[연홍이창/금자희재] 인因 (1)\n', '\n[금자희재/희재금자] 내 상사가 사람을 꼬시려고 해 (1)\n', '\n[금자희재] Daybreak\n', '\n[금자희재/희재금자] 아주 보통의 야유회\n', '\n약約 上\n', '\n습작2\n', '\n[금자희재] 도리와 사랑의 차이\n', '\n어떤 사랑은 아직 쓰이지 않아서\n', '\n결혼\n', '\n어느 하루\n', '\n[금자희재] 대소동\n', '\n[금자희재] 그림1\n', '\nPlace of pride\n', '\n[금자희재/희재금자] 비포 미드나잇\n', '\n하이에나 (2)\n']입니다.
3 페이지 작품 제목 리스트는 ['\n하이에나 (1)\n', '\n[금자희재] HB\n', '\n[금자희재/희재금자] 비포 선라이즈\n', '\n[금자희재/희재금자] 세상에서 제일 어려운 건\n', '\n충忠\n', '\n[금자희재/희재금자] 0310\n', '\n[금자희재] 드레스를 입은 밤\n', 

### 함수 형식으로 정리

In [4]:
def popularity_filter_crawling(couple_name, page):
    
    driver = webdriver.Chrome('chromedriver.exe')
    url = 'https://www.postype.com/search?page={}&keyword='+ couple_name +'&period=any_time&sort=popularity&options_title=1&options_sub_title=1&options_content=1&options_tags=1&options_nickname=1'

    # 데이터프레임을 생성하기 위한 리스트 생성
    title_li=[] # 글 제목
    url_li=[] # 글 URL
    writer_li = [] # 작가 이름
    like_li=[] # 좋아요 수 


    # 크롤링 함수 만들기 
    for i in range(1, page):
        link = url.format(i)
        driver.get(link)

        time.sleep(3)

        html=requests.get(link).content
        soup=BeautifulSoup(html, 'html.parser')
        # print(soup)

        res=soup.find_all('h3', {'class': 'pst-title'}) # 제목 텍스트를 담고 있는 a 태그의 부모 h3 태그 전부 꺼내기
        res2=soup.find_all('div', {'class' : 'pst-action'}) # 좋아요 수를 담은 span 태그를 답고 있는 div 태그 전부 꺼내기
        res3=soup.find_all('div', {'class' : 'pst-blog'})

        for l in res:
            title=l.get_text() # 제목 텍스트 추출
            text_url=l.find('a')['href'] # 글 링크 추출
            title_li.append(title) # 제목 리스트에 추가
            url_li.append(text_url) # 링크 리스트에 추가

        print('{} 페이지 작품 제목 리스트는 {}입니다.'.format(i, title_li))

        for w in res3:
            writer_name=w.find('a').get_text() # a 태그를 찾지 않고 텍스트를 추출할 경우 블로그 이름까지 추출된다
            writer_li.append(writer_name)

        for h in res2:
            like_count = h.find('span', {'class' : 'count'})

            if like_count!=None: # 좋아요가 있는 게시물
                like_count=int(like_count.get_text())
                like_li.append(like_count)

            else: # 좋아요가 없는 게시물
                like_li.append(0) # 0을 추가한다


    # 리스트 값들의 좌우 공백 삭제            
    title_li = [t.strip() for t in title_li] 
    writer_li=[x.strip() for x in writer_li]

    print('크롤링이 완료되었습니다. 데이터로 저장합니다.')
    driver.quit() # 가상 브라우저 종료 


    # 데이터 프레임으로 만들기
    df = pd.DataFrame(data={'title' : title_li, 
                        'writer' : writer_li,
                        'url' : url_li,
                       'likes' : like_li})
    df.to_csv('postype_crawling.csv', encoding='cp949') # 글자가 깨지지 않게 하려면 인코딩을 cp949로 해야 한다!
    
    return df

In [5]:
couple_name=input()
page= int(input())

popularity_filter_crawling(couple_name, page)

금자희재
5
1 페이지 작품 제목 리스트는 ['\n하이에나 (1)\n', '\n[금자희재] HB\n', '\n[금자희재/희재금자] 비포 선라이즈\n', '\n[금자희재/희재금자] 세상에서 제일 어려운 건\n', '\n충忠\n', '\n[금자희재/희재금자] 0310\n', '\n[금자희재] 드레스를 입은 밤\n', '\n[금자희재/희재금자] 사계 (1)\n', '\n금자의 세계\n', '\n[연홍이창/금자희재] 인因 (1)\n', '\n[금자희재/희재금자] 내 상사가 사람을 꼬시려고 해 (1)\n', '\n[금자희재] Daybreak\n']입니다.
2 페이지 작품 제목 리스트는 ['\n하이에나 (1)\n', '\n[금자희재] HB\n', '\n[금자희재/희재금자] 비포 선라이즈\n', '\n[금자희재/희재금자] 세상에서 제일 어려운 건\n', '\n충忠\n', '\n[금자희재/희재금자] 0310\n', '\n[금자희재] 드레스를 입은 밤\n', '\n[금자희재/희재금자] 사계 (1)\n', '\n금자의 세계\n', '\n[연홍이창/금자희재] 인因 (1)\n', '\n[금자희재/희재금자] 내 상사가 사람을 꼬시려고 해 (1)\n', '\n[금자희재] Daybreak\n', '\n[금자희재/희재금자] 아주 보통의 야유회\n', '\n약約 上\n', '\n습작2\n', '\n[금자희재] 도리와 사랑의 차이\n', '\n어떤 사랑은 아직 쓰이지 않아서\n', '\n결혼\n', '\n어느 하루\n', '\n[금자희재] 대소동\n', '\n[금자희재] 그림1\n', '\nPlace of pride\n', '\n[금자희재/희재금자] 비포 미드나잇\n', '\n하이에나 (2)\n']입니다.
3 페이지 작품 제목 리스트는 ['\n하이에나 (1)\n', '\n[금자희재] HB\n', '\n[금자희재/희재금자] 비포 선라이즈\n', '\n[금자희재/희재금자] 세상에서 제일 어려운 건\n', '\n충忠\n', '\n[금자희재/희재금자] 0310\n', '\n[금자희재] 드레스를 입은

,title,writer,url,likes
0,하이에나 (1),칸,https://hykanena.postype.com/post/6574551,61
1,[금자희재] HB,이즈,https://if-yourworld-fallsapart.postype.com/po...,61
2,[금자희재/희재금자] 비포 선라이즈,시월,https://paganiniana.postype.com/post/6167155,60
3,[금자희재/희재금자] 세상에서 제일 어려운 건,시월,https://paganiniana.postype.com/post/6152100,58
4,충忠,SAND,https://ampersand0905.postype.com/post/6460304,57
5,[금자희재/희재금자] 0310,시월,https://paganiniana.postype.com/post/6107554,57
6,[금자희재] 드레스를 입은 밤,이즈,https://if-yourworld-fallsapart.postype.com/po...,54
7,[금자희재/희재금자] 사계 (1),시월,https://paganiniana.postype.com/post/6130292,54
8,금자의 세계,윤나무,https://bubbledreamer.postype.com/post/6482887,53
9,[연홍이창/금자희재] 인因 (1),이즈,https://if-yourworld-fallsapart.postype.com/po...,52


## 인기작을 많이 쓴 작가 추려내기
장르별로 인기 작가들이 포진되어 있듯이, 2차 창작에서도 인기 작가들이 있다. 작품 중 인기작이 많은 작가들을 순위별로 나열하고, 해당 블로그에서 사용자가 검색한 커플링의 글들을 인기순으로 검색하여 보여준다. 

위에서 미리 다운로드 해 놓은 데이터프레임을 이용해 이를 구현해본다.

In [2]:
# 데이터 불러오기
df = pd.read_csv('postype_crawling.csv', encoding='cp949', index_col=0) # encoding을 빼먹지 않도록 한다!
df.head(10)

,title,writer,url,likes
0,하이에나 (1),칸,https://hykanena.postype.com/post/6574551,61
1,[금자희재] HB,이즈,https://if-yourworld-fallsapart.postype.com/po...,61
2,[금자희재/희재금자] 비포 선라이즈,시월,https://paganiniana.postype.com/post/6167155,60
3,[금자희재/희재금자] 세상에서 제일 어려운 건,시월,https://paganiniana.postype.com/post/6152100,58
4,충忠,SAND,https://ampersand0905.postype.com/post/6460304,57
5,[금자희재/희재금자] 0310,시월,https://paganiniana.postype.com/post/6107554,57
6,[금자희재] 드레스를 입은 밤,이즈,https://if-yourworld-fallsapart.postype.com/po...,54
7,[금자희재/희재금자] 사계 (1),시월,https://paganiniana.postype.com/post/6130292,54
8,금자의 세계,윤나무,https://bubbledreamer.postype.com/post/6482887,53
9,[연홍이창/금자희재] 인因 (1),이즈,https://if-yourworld-fallsapart.postype.com/po...,52


In [3]:
# 인기작가 선별
# 기준은 크롤링된 데이터에 포함된 작품의 수 비교

writer_data = dict(df['writer'].value_counts()) # dict로 변경
writer_li = list(writer_data.keys()) # key만 추출해 list화
top5 = writer_li[:5] # 인기작가 TOP 5

In [4]:
url_li[0] = url_li[0][:len(url_li[0])-12]
url_li[0]

NameError: name 'url_li' is not defined

In [5]:
# 블로그의 글 중 '금자희재' 커플링에 대한 글 검색 & 인기순으로 sort
# 블로그 링크 추출
top5_df = df[df['writer'].isin(top5)] # top5 작가들의 작품만 뽑아내기
url_li = list(top5_df['url']) 

top5_url_li=[] # url 리스트
for url in url_li:
    url=url[:len(url)-12] # 작품번호 7자리와 'post' 제거
    print(url)
    top5_url_li.append(url)
    
top5_url_li = set(top5_url_li) # 중복 주소 제거
print(top5_url_li)

https://if-yourworld-fallsapart.postype.com/
https://paganiniana.postype.com/
https://paganiniana.postype.com/
https://ampersand0905.postype.com/
https://paganiniana.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://paganiniana.postype.com/
https://bubbledreamer.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://paganiniana.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://paganiniana.postype.com/
https://ampersand0905.postype.com/
https://ampersand0905.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://bubbledreamer.postype.com/
https://ampersand0905.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://ampersand0905.postype.com/
https://paganiniana.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://paganiniana.postype.com/
https://paganiniana.postype.com/
https://if-yourworld-fallsapart.postype.com/
https://ampersand0905.postype.com/
https://bubbledreamer.postype.com/
https://ampersand0905.postyp

url을 추출한 것까지는 좋았지만 작가명과 연결하기가 번거롭다. 다른 방법을 시도해본다.

In [6]:
# 인기 작가명-블로그 url 연결하기
top5_df = df[df['writer'].isin(top5)] # top5 작가들의 작품만 뽑아내기
top5_df.head(5)

,title,writer,url,likes
1,[금자희재] HB,이즈,https://if-yourworld-fallsapart.postype.com/po...,61
2,[금자희재/희재금자] 비포 선라이즈,시월,https://paganiniana.postype.com/post/6167155,60
3,[금자희재/희재금자] 세상에서 제일 어려운 건,시월,https://paganiniana.postype.com/post/6152100,58
4,충忠,SAND,https://ampersand0905.postype.com/post/6460304,57
5,[금자희재/희재금자] 0310,시월,https://paganiniana.postype.com/post/6107554,57


In [7]:
# writer 열을 기준으로 중복만 제거
top5_df = top5_df.drop_duplicates(['writer'])
top5_df

,title,writer,url,likes
1,[금자희재] HB,이즈,https://if-yourworld-fallsapart.postype.com/po...,61
2,[금자희재/희재금자] 비포 선라이즈,시월,https://paganiniana.postype.com/post/6167155,60
4,충忠,SAND,https://ampersand0905.postype.com/post/6460304,57
8,금자의 세계,윤나무,https://bubbledreamer.postype.com/post/6482887,53
39,[금자희재] 금자는 희재를 좋아한다.,자염,https://rubeolux.postype.com/post/6123107,26


In [8]:
# 블로그 url 재추출
url_li = list(top5_df['url'])

top5_url_li=[]
for u in url_li:
    u=u[:len(u)-12] # 작품번호 7자리와 'post' 제거
#     print(u)
    top5_url_li.append(u)
    
top5_url_li

['https://if-yourworld-fallsapart.postype.com/',
 'https://paganiniana.postype.com/',
 'https://ampersand0905.postype.com/',
 'https://bubbledreamer.postype.com/',
 'https://rubeolux.postype.com/']

In [9]:
# 작가 & 블로그 링크 딕셔너리 생성
wb_li = dict(zip(top5, top5_url_li))
wb_li

{'이즈': 'https://if-yourworld-fallsapart.postype.com/',
 '시월': 'https://paganiniana.postype.com/',
 'SAND': 'https://ampersand0905.postype.com/',
 '윤나무': 'https://bubbledreamer.postype.com/',
 '자염': 'https://rubeolux.postype.com/'}

### 블로그에서 특정 커플링 인기순으로 검색, 크롤링
블로그 내 검색 후 검색 결과를 크롤링한 후, 그 결과를 데이터프레임으로 만들어 좋아요 기준으로 재배열한다.

In [13]:
# 블로그 내 검색 url
couple_name = '금자희재'
url = top5_url_li[0] + 'search?page={}&keyword=' + couple_name

driver = webdriver.Chrome('chromedriver.exe')

# 넘길 페이지 수 지정
page = 2

# 데이터프레임을 생성하기 위한 리스트 생성
title_li=[] # 글 제목
url_li=[] # 글 URL
like_li=[] # 좋아요 수 


# 크롤링 함수 만들기 

for i in range(1, page):
    link = url.format(i)
    driver.get(link)
    
    time.sleep(3)
    
    html=requests.get(link).content
    soup=BeautifulSoup(html, 'html.parser')
    # print(soup)

    res=soup.find_all('div', {'class' : 'article-media-body'}) # 좋아요 수를 담은 span 태그를 답고 있는 div 태그 전부 꺼내기
    res2=soup.find_all('a', {'class' : 'btn-like'})

    for l in res:
        title=l.find('h5').get_text()
        title_li.append(title) # 제목 리스트에 추가
        
    text_url=soup.select(".article-media-body > .article-content,.has-snippet > a")
    for u in text_url:
        page_url = u['href']
        url_li.append(page_url)
    
    print('{} 페이지 작품 제목 리스트는 {}입니다.'.format(i, title_li))
    print('{} 페이지 작품 url 리스트는 {}입니다.'.format(i, url_li))


    for h in res2:
        like_count = h.find('span', {'class' : 'count'})

        if like_count!=None: # 좋아요가 있는 게시물
            like_count=int(like_count.get_text())
            like_li.append(like_count)

        else: # 좋아요가 없는 게시물
            like_li.append(0) # 0을 추가한다
    print(like_li)

            
# 리스트 값들의 좌우 공백 삭제            
title_li = [t.strip() for t in title_li] 

# 리스트 값 확인
print(len(title_li))
print(len(writer_li))
print(len(like_li))

print('크롤링이 완료되었습니다. 데이터로 저장합니다.')
driver.quit() # 가상 브라우저 종료 


# 데이터 프레임으로 만들기
df = pd.DataFrame(data={'title' : title_li, 
                    'url' : url_li,
                   'likes' : like_li})

df

1 페이지 작품 제목 리스트는 ['[금자희재] 무언가', '[무택마담] 있을 법한 이야기', '[금자희재] 자전거', '[무택마담] 들려주는 이야기', '[금자희재] If The World Was Ending', '[금자희재] Blood//Water', '[연홍이창] 인因(5)', '[금자희재] Our Shining Days', '[연홍이창] 인因(4)', '연홍이창 그림 백업', '[금자희재] Mermaid', '[금자희재] 밀회']입니다.
1 페이지 작품 url 리스트는 ['https://if-yourworld-fallsapart.postype.com/post/7743837', 'https://if-yourworld-fallsapart.postype.com/post/7682780', 'https://if-yourworld-fallsapart.postype.com/post/7722764', 'https://if-yourworld-fallsapart.postype.com/post/7630229', 'https://if-yourworld-fallsapart.postype.com/post/7588018', 'https://if-yourworld-fallsapart.postype.com/post/7541193', 'https://if-yourworld-fallsapart.postype.com/post/7501498', 'https://if-yourworld-fallsapart.postype.com/post/6953868', 'https://if-yourworld-fallsapart.postype.com/post/7291876', 'https://if-yourworld-fallsapart.postype.com/post/7311310', 'https://if-yourworld-fallsapart.postype.com/post/7033559', 'https://if-yourworld-fallsapart.postype.com/post/7012053']입니다.
[11, 14

,title,url,likes
0,[금자희재] 무언가,https://if-yourworld-fallsapart.postype.com/po...,11
1,[무택마담] 있을 법한 이야기,https://if-yourworld-fallsapart.postype.com/po...,14
2,[금자희재] 자전거,https://if-yourworld-fallsapart.postype.com/po...,16
3,[무택마담] 들려주는 이야기,https://if-yourworld-fallsapart.postype.com/po...,13
4,[금자희재] If The World Was Ending,https://if-yourworld-fallsapart.postype.com/po...,16
5,[금자희재] Blood//Water,https://if-yourworld-fallsapart.postype.com/po...,13
6,[연홍이창] 인因(5),https://if-yourworld-fallsapart.postype.com/po...,24
7,[금자희재] Our Shining Days,https://if-yourworld-fallsapart.postype.com/po...,15
8,[연홍이창] 인因(4),https://if-yourworld-fallsapart.postype.com/po...,20
9,연홍이창 그림 백업,https://if-yourworld-fallsapart.postype.com/po...,14


데이터 저장에 성공했다. 하지만 인기순으로 정렬되지는 않았다. 코드를 살짝 고쳐서 데이터가 인기순으로 나열되도록 해보자.

In [14]:
# 블로그 내 검색 url
couple_name = '금자희재'
url = top5_url_li[0] + 'search?page={}&keyword=' + couple_name # 1순위 작가 작품만

driver = webdriver.Chrome('chromedriver.exe')

# 넘길 페이지 수 지정
page = 2

# 데이터프레임을 생성하기 위한 리스트 생성
title_li=[] # 글 제목
url_li=[] # 글 URL
like_li=[] # 좋아요 수 


# 크롤링 함수 만들기 

for i in range(1, page):
    link = url.format(i)
    driver.get(link)
    
    time.sleep(3)
    
    html=requests.get(link).content
    soup=BeautifulSoup(html, 'html.parser')
    # print(soup)

    res=soup.find_all('div', {'class' : 'article-media-body'}) # 좋아요 수를 담은 span 태그를 답고 있는 div 태그 전부 꺼내기
    res2=soup.find_all('a', {'class' : 'btn-like'})

    for l in res:
        title=l.find('h5').get_text()
        title_li.append(title) # 제목 리스트에 추가
        
    text_url=soup.select(".article-media-body > .article-content,.has-snippet > a")
    for u in text_url:
        page_url = u['href']
        url_li.append(page_url)
        
    
    print('{} 페이지 작품 제목 리스트는 {}입니다.'.format(i, title_li))
    print('{} 페이지 작품 url 리스트는 {}입니다.'.format(i, url_li))


    for h in res2:
        like_count = h.find('span', {'class' : 'count'})

        if like_count!=None: # 좋아요가 있는 게시물
            like_count=int(like_count.get_text())
            like_li.append(like_count)

        else: # 좋아요가 없는 게시물
            like_li.append(0) # 0을 추가한다
    print(like_li)

            
# 리스트 값들의 좌우 공백 삭제            
title_li = [t.strip() for t in title_li] 

print('크롤링이 완료되었습니다. 데이터로 저장합니다.')
driver.quit() # 가상 브라우저 종료 


# 데이터 프레임으로 만들기
df = pd.DataFrame(data={'title' : title_li, 
                    'url' : url_li,
                   'likes' : like_li})

# 인기순으로 정렬하기
df = df.sort_values(by=['likes'], ascending=False) # likes를 기준으로 내림차순

df.to_csv('블로그 내 인기순 리스트.csv', encoding='cp949')

1 페이지 작품 제목 리스트는 ['[금자희재] 무언가', '[무택마담] 있을 법한 이야기', '[금자희재] 자전거', '[무택마담] 들려주는 이야기', '[금자희재] If The World Was Ending', '[금자희재] Blood//Water', '[연홍이창] 인因(5)', '[금자희재] Our Shining Days', '[연홍이창] 인因(4)', '연홍이창 그림 백업', '[금자희재] Mermaid', '[금자희재] 밀회']입니다.
1 페이지 작품 url 리스트는 ['https://if-yourworld-fallsapart.postype.com/post/7743837', 'https://if-yourworld-fallsapart.postype.com/post/7682780', 'https://if-yourworld-fallsapart.postype.com/post/7722764', 'https://if-yourworld-fallsapart.postype.com/post/7630229', 'https://if-yourworld-fallsapart.postype.com/post/7588018', 'https://if-yourworld-fallsapart.postype.com/post/7541193', 'https://if-yourworld-fallsapart.postype.com/post/7501498', 'https://if-yourworld-fallsapart.postype.com/post/6953868', 'https://if-yourworld-fallsapart.postype.com/post/7291876', 'https://if-yourworld-fallsapart.postype.com/post/7311310', 'https://if-yourworld-fallsapart.postype.com/post/7033559', 'https://if-yourworld-fallsapart.postype.com/post/7012053']입니다.
[11, 14

## 인기글 첫 문단만 크롤링해 추가하기
> 아무리 좋아하는 커플링의 인기글이라고 해도 취향과 맞지 않을 수 있다. 물론 소개글이나 부제목을 보고 계속 읽을지 말지를 선택할 수도 있지만, 보통은 첫 문단에서 결정이 난다. BeautifulSoup을 통해 첫 문단만 크롤링해보자. 단, 글 내용과 상관없는 텍스트가 크롤링되면 곤란하므로 기준을 세운다. 


- '-', '*' 같은 기호가 붙지 않는다.
- 한글로 시작된다. 

또, 포스타입은 자연스러운 줄바꿈이 아니면 각각 다른 p 태그에 속해있으므로, 사용자에게 조금이나마 더 많은 정보를 제공하기 위해서는 크롤링할 p 태그의 갯수를 정해두는 편이 좋을 것이다.

In [15]:
# 시험해볼 데이터
df

,title,url,likes
6,[연홍이창] 인因(5),https://if-yourworld-fallsapart.postype.com/po...,24
11,[금자희재] 밀회,https://if-yourworld-fallsapart.postype.com/po...,24
10,[금자희재] Mermaid,https://if-yourworld-fallsapart.postype.com/po...,23
8,[연홍이창] 인因(4),https://if-yourworld-fallsapart.postype.com/po...,20
2,[금자희재] 자전거,https://if-yourworld-fallsapart.postype.com/po...,16
4,[금자희재] If The World Was Ending,https://if-yourworld-fallsapart.postype.com/po...,16
7,[금자희재] Our Shining Days,https://if-yourworld-fallsapart.postype.com/po...,15
1,[무택마담] 있을 법한 이야기,https://if-yourworld-fallsapart.postype.com/po...,14
9,연홍이창 그림 백업,https://if-yourworld-fallsapart.postype.com/po...,14
3,[무택마담] 들려주는 이야기,https://if-yourworld-fallsapart.postype.com/po...,13


In [26]:
# 각 page url 리스트로 만들기
page_url_li = df['url'].tolist()
# page_url_li


text_li=[]
# 각 page 별 문단 크롤링하기
try:
    for i in page_url_li:
        text_link = i
        html=requests.get(text_link).content
        soup=BeautifulSoup(html, 'html.parser')

        test=soup.select('#post-content > p')
        for j in range(10):
            p=test[j].text
            text_li.append(p)
            
except IndexError:
    print('해당 게시물은 소설이 아닙니다.')

    
print(text_li)

해당 게시물은 소설이 아닙니다.
['악토버- Cherry Blossom\xa0', '', '', '', '연홍은 혹 세자가 기방에 있다는 소문이 돌 세라 손님들이 드나드는 정문이 아닌 기녀들의 숙소와\xa0하인들만이 주로 드나드는 안채 옆의 쪽문을 사용했다. 자초지종을 설명하니 세자와 좌익위가 수긍했기에 망정이었다. 일국의 세자가 이리 초라한 문을 드나들어 봤겠냐만은. 손님들을 맞을 준비로 분주한 와중 힐끔거리며 두 사내를 살피는 기녀들의 눈초리가 따가웠다. 또 함구시킬 것이 늘어났다는 생각에 연홍은 그저 한숨만 늘었다. 기녀들이 기거하는 숙소에는 차마 세자를 안내하지 못하겠던 홍은 창을 자신이 기거하는 안채로 안내했다. 루주의 방은 다행히 부끄럽지 않을 정도는 되었기에.', '', '비를 피하고 안으로 들어오니 젖은 옷이 고민이었다. 연홍은 조심스럽게 세자와 좌익위에게 탕을 준비해 드릴지 여부를 물었다. 장소가 장소이고 상대의 신분이 신분인지라 혹 다른 의미로 받아들여질까 저어되었던 탓이다. 다행히 세자와 좌익위는 별다른 곡해 없이 긍정의 답을 주었고 연홍은 사람을 불러 탕에 따뜻한 물을 받아 두라 일렀다. 그러고는 그들이 입을 새 옷을 준비하였는데, 창의 몸에 맞는 도포를 찾지 못해 손목으로부터 반 뼘이나 부족한 소매를 입게 되었다는 것은 웃지 못할 일일 것이다. 창의 고름을 매주던 홍은 비실비실 새어 나오는 미소를 드러냈고 옆에서 좌익위는 신나게 폭소를 터뜨렸지만 말이다.', '석반 전에는 환궁하리라 여겼건만 시각이 늦어 홍은 상을 내왔고,\xa0수라상만큼은 아니나 상다리가 휘어질 정도로 내온 상에 창은 만족스럽게 배를 채웠다. 비에 젖은 후 따뜻한 물에 담가 여독이 풀린 몸과 부른 배, 후식으로 나온 맑은 차에 노곤하니 눈꺼풀이 무거워진다. 그러고 보니 예의 그 독살 사건이 있고 난 뒤로 끼니도 제대로 챙기지 못하고 밤잠도 설쳤더랬지. 긴장이 풀린 탓인가. 창은 눈을 감았다. 궁에서는 없을 평온한 휴식이었다.\xa0', '', '얼마나 잠이 들었을까. 

In [ ]:
# 참고
# https://m.blog.naver.com/PostView.nhn?blogId=owl6615&logNo=221518357627&proxyReferer=https:%2F%2Fwww.google.com%2F
# https://beomi.github.io/2017/02/27/HowToMakeWebCrawler-With-Selenium/
# https://qastack.kr/programming/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
# https://www.python2.net/questions-241649.htm